In [1]:
from collections import namedtuple, deque
from types import SimpleNamespace
from contextlib import contextmanager
import itertools
import os

from tqdm import tqdm
import numpy as np
import scipy as sp
import scipy.signal as signal
import scipy.misc
import matplotlib.pyplot as plt
import cv2
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as trans

import vmdata
import more_trans
import more_sampler
import exprlib
import utils

from more_sampler import SlidingWindowBatchSampler

%matplotlib inline

In [2]:
import pdb

In [3]:
feature_params = {
    'blockSize': 7,
    'maxCorners': 100,
    'minDistance': 7,
    'qualityLevel': 0.3,
    'mask': None,
}
lk_params = {
    'criteria': (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
}
prmy_params = {
    'winSize': (15, 15),
    'maxLevel': 8,
}
fb_params = (
    0.5,
    1,
    15,
    3,
    5,
    1.2,
    0,
)

In [4]:
root = vmdata.prepare_dataset_root(9, (8, 0, 0))
#normalize = trans.Normalize(*vmdata.get_normalization_stats(root, bw=True))

basedir = 'data.experiments-cv2flow/multiscale'
os.makedirs(basedir, exist_ok=True)


class MultiscaleFlowLauncher(exprlib.ExperimentLauncher):
    def __init__(self):
        super().__init__(basedir, prefix='flow_', suffix='.npz',
                         translate_kwargs={
                             'root': str,
                             'indices': list,
                             'prmy_params': dict,
                             'fb_params': list,
                         })
    def load_result(self, filename):
        data = np.load(filename)
        flows = [data[str(j)] for j in range(len(data.keys()))]
        del data
        return flows,
    
    def store_result(self, filename, *args):
        scales_flows, = args
        scales_flows = dict((str(j), f) for j, f in enumerate(scales_flows))
        np.savez(filename, **scales_flows)
    
    def run(self, **kwargs):
        root = kwargs['root']
        indices = kwargs['indices']
        prmy_params = kwargs['prmy_params']
        fb_params = kwargs['fb_params']
        
        flows = []
        window = deque(maxlen=2)
        with vmdata.VideoDataset(root) as vdset:
            for j in indices:
                im = vdset[j]
                im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
                pr = cv2.buildOpticalFlowPyramid(im, **prmy_params)[1][::2]
                pr = [l[..., np.newaxis] for l in pr]
                window.append(pr)
                
                if len(window) < 2:
                    continue
                
                for k, (i1, i2) in enumerate(zip(*window)):
                    f = cv2.calcOpticalFlowFarneback(i1, i2, None, *fb_params)
                    while k >= len(flows):
                        flows.append([])
                    flows[k].append(f)
        flows = list(map(np.stack, flows))
        return flows,

In [9]:
def time_averaged(flows, width=3):
    """
    :param flows: of shape (N, H, W, 2)
    :param window: the sliding window width along the first axis of ``flows``
    """
    kernel = np.ones((width, 1))
    sh = flows.shape
    flows = flows.reshape((flows.shape[0], -1))
    flows = signal.convolve2d(flows, kernel, mode='valid')
    flows = flows.reshape((-1, *sh[1:]))
    return flows

def downsampled(flow, scale=0.5):
    flow = scipy.misc.imresize(flow, size=scale)
    return flow


def draw_vector_field(flow, figsize, scale=0.5, bw=False):
    fl = np.stack((downsampled(flow[..., 0], scale=scale),
                   downsampled(flow[..., 1], scale=scale)), axis=2)
    fig, ax = plt.subplots(figsize=figsize)
    with exprlib.fig_as_data(plt, fig, ax) as ns:
        ax.quiver(np.arange(fl.shape[1]),
                  np.arange(fl.shape[0])[::-1],
                  fl[..., 0], fl[..., 1])
    data = ns.data
    if bw:
        data = cv2.cvtColor(data, cv2.COLOR_RGB2GRAY)
    return data

def crop_white_border(img):
    """
    Crop white border from B&W image of shape (H, W).
    """
    img = np.max(img, axis=2)
    rpixels = np.nonzero(np.min(img, axis=0) < 255)[0]
    rmin, rmax = np.min(rpixels), np.max(rpixels)
    cpixels = np.nonzero(np.min(img, axis=1) < 255)[0]
    cmin, cmax = np.min(cpixels), np.max(cpixels)
    img = img[cmin:1+cmax,rmin:1+rmax]
    return img

In [6]:
def visualize_flows(flows, filename, vdset_indices=None, title='', fps=3):
    """
    Visualize flows in arrows.
    
    :param flows: flow data of shape (N, H, W, 2)
    :param filename: video file to write to
    :param vdset_indices: tuple of (vdset, indices) used in
           MultiscaleFlowLauncher; if provided, put the original frame
           on the left of the flow visualization
    """
    def _visualize_flow_once(vdset, fid, fl):
        datas = draw_vector_field(fl, (9, 4), scale=0.2, bw=True)
        datal = draw_vector_field(fl, (9, 4), scale=0.05, bw=True)

        #datas = crop_white_border(datas)
        #datal = crop_white_border(datal)

        canvas = np.zeros((datas.shape[0] + 2 + datal.shape[0],
                           np.max([datas.shape[1], datal.shape[1]]))).astype(np.uint8)
        canvas[:datas.shape[0],:datas.shape[1]] = datas
        canvas[-datal.shape[0]:,:datal.shape[1]] = datal

        if fid is not None:
            origf = cv2.cvtColor(vdset[fid], cv2.COLOR_RGB2GRAY)
            aug_canvas = np.zeros((max(origf.shape[0], canvas.shape[0]),
                                   origf.shape[1] + canvas.shape[1])).astype(np.uint8)
            aug_canvas[:origf.shape[0], :origf.shape[1]] = origf
            aug_canvas[:canvas.shape[0], -canvas.shape[1]:] = canvas
            canvas = aug_canvas
        
        canvas = canvas.astype(np.uint8)
        return canvas
    
    
    if not flows.shape[0]:
        return
    
    flows = time_averaged(flows, width=3)
    
    try:
        vdset, indices = vdset_indices
    except TypeError:
        vdset = None
    else:
        indices = indices[1:-2]
        assert len(indices) == flows.shape[0], '{} != {}'.format(len(indices), flows.shape[0])
    
    wh = tuple(_visualize_flow_once(vdset, 1, flows[0]).shape[:2][::-1])
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    if not title:
        title = 'flow'
        
    with utils.videowritercontext(filename, fourcc, fps, wh) as outfile:
        flows_it = zip(indices, flows) if vdset_indices \
                   else zip(itertools.repeat(None), flows)
        for fid, fl in tqdm(flows_it, total=flows.shape[0], ascii=True):
            sh = fl.shape[:2]
            canvas = _visualize_flow_once(vdset, fid, fl)
            canvas = np.repeat(canvas[...,np.newaxis], 3, axis=2)
            outfile.write(canvas)

In [7]:
def visualize_flows_hsv(flows, filename, title='', fps=3):
    """
    Visualize flows in HSV.
    
    :param flows: flow data of shape (N, H, W)
    :param filename: video file to write to
    """
    if not flows.shape[0]:
        return
    hw = flows[0].shape[:2]
    wh = tuple(hw[::-1])
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    if not title:
        title = 'flow'
    
    hsv = np.zeros(flows[0].shape[:2] + (3,), dtype=np.uint8)
    hsv[..., 1] = 255
    with utils.videowritercontext(filename, fourcc, fps, wh) as outfile:
        for f in tqdm(flows, total=flows.shape[0], ascii=True):
            mag, ang = cv2.cartToPolar(f[..., 0], f[..., 1])
            hsv[..., 0] = ang * 180 / np.pi / 2
            hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
            rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
            outfile.write(rgb)

In [8]:
def visualize_flows_all_scales(vmethod, flows_scales, basefilename, fps=3, **kwargs):
    for s, flows in tqdm(enumerate(flows_scales), total=len(flows_scales), ascii=True):
        filename = basefilename + '.sc{}.avi'.format(s)
        tqdm.write('Beginning {} at scale {}'.format(vmethod.__name__, s))
        vmethod(flows, filename, title='flow_scale{}'.format(s), fps=fps, **kwargs)

In [ ]:
launcher = MultiscaleFlowLauncher()
indices = range(1000)
flows, = launcher(root=root, indices=indices,
                  prmy_params=prmy_params, fb_params=fb_params)

In [ ]:
with vmdata.VideoDataset(root) as vdset:
    visualize_flows_all_scales(visualize_flows, flows, launcher.result_filename,
                               vdset_indices=(vdset, indices))

In [15]:
#visualize_flows_all_scales(flows, launcher.result_filename)

Conclusion:
Gunnar Farneback dense optical flow produces worse quality under individual scale than in a pyramid of scales.
When the scale is small, dolphin is polluted by water movements.
When the scale is large, dolphin, when far from camera, is not captured.